In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
%run test_functions.ipynb

#Todo
"""
1. We need to fix this code after changing K_test function on test_functions.py
2. Change x-axis of K_test plot to computation time.
"""

"""

Plot 3: Testing convergence on different K value

Here we run for fixed T_cap, However we use optimal step-size for each case with very low epsilon 1e-8, so as to not
terminate before T_cap. 

"""


current_path = os.getcwd()
parent_path = os.path.dirname(current_path)


# T = 100000 #Number of Total Iteration
J_sst = 10  # Number of Cohorts
L_sst = 25  # Number of Treatments


dual_gap_freq_sst = 10
n_sst = 10000
K_list_sst = [10,30,50,75,100,150,200]  # 6 Instance
T_cap_sst = 10000
print_opt = 1

repeats_sst = 50  # Currently we don't take average here
m_sst = 5

# stat_list_sst[n_idx][rep_idx][alg_idx]

result_list = []

for rep_idx in range(repeats_sst):
    stat_list = K_test_iter(J_sst, L_sst, dual_gap_freq_sst, n_sst, K_list_sst, T_cap_sst,
                                m_sst, print_opt)
    result_list.append(stat_list)

#Save this data into csv form

#Get the length of each dual_gap_list
dual_len = len(result_list[0][0].dual_gap_list[0])
dual_gap_arr = np.zeros((repeats_sst,len(K_list_sst)+1, dual_len))

for rep_idx in range(repeats_sst):
    for k_idx in range(len(K_list_sst)+1):
        dual_gap_arr[rep_idx,k_idx,:] = result_list[rep_idx][k_idx].dual_gap_list[0]

avg_dual_gap = np.average(dual_gap_arr,axis = 0)

#Save in csv

data_list = []
for k_idx in range(len(K_list_sst)):
    temp_list = [n_sst, J_sst * L_sst, m_sst,K_list_sst[k_idx], T_cap_sst, dual_gap_freq_sst]
    temp_list.append(avg_dual_gap[k_idx,:].tolist())
    data_list.append(temp_list)
temp_list = [n_sst, J_sst * L_sst, m_sst,'FMD', T_cap_sst, dual_gap_freq_sst]
temp_list.append(avg_dual_gap[len(K_list_sst),:].tolist())
data_list.append(temp_list)

df= pd.DataFrame(data_list, columns = ['n','d','m','K','Total_Iter', 'Dual_Freq', 'Avg_Dual_Gap'])
save_path = parent_path + '/results/K_test0_n=' +str(n_sst) + 'd=' + str(J_sst * L_sst) + '_m=' + str(m_sst) + '.csv'
df.to_csv(save_path, index=False)

In [ ]:
#
# i_flag_count_list = []
#
# for n_idx in range(len(n_list_sst)):
#     temp_list = []
#     for alg_idx in range(len(alg_list_sst)):
#         temp = np.zeros(repeats_sst)
#         for rep_idx in range(repeats_sst):  # repeats_sst == 1 code
#             temp[rep_idx] = stat_list_sst[n_idx][rep_idx][alg_idx].i_flag_count
#         temp_list.append(np.mean(temp))
#     i_flag_count_list.append(temp_list)
#
# for n_idx in range(len(n_list_sst)):
#     print('i flag count when n = %s' % n_list_sst[n_idx])
#     for k_idx, value in enumerate(i_flag_count_list[n_idx]):
#         print('K value: %s, i_flag_count: %s ' % (K_list_sst[k_idx], value))
#
# df_dicts = {}
#
# for index, key in np.ndenumerate(n_list_sst):
#     df_dicts[key] = i_flag_count_list[index]
#
# cols = [10, 100, 500, 1000, 'n']
#
# print("Comparison table")
#
# comparison_table = pd.DataFrame(df_dict)
#
# comparison_table = comparison_table[cols]  # Sort table columns
# comparison_table.to_csv('results/Ktest.csv', index=False)
# print(comparison_table)
#
# color_list_sst = []
# alg_name_sst = []
# for k_idx in range(len(K_list_sst)):
#     if K_list_sst[k_idx] != 'i_star':
#         alg_name_sst.append('K = ' + str(K_list_sst[k_idx]))
#     else:
#         alg_name_sst.append('K = Full Sample')
#
# avg_dual_gap = []  # avg_dual_gap[n_idx][k_idx]
#
# for n_idx in range(len(n_list_sst)):
#     temp_list = []
#     for alg_idx in range(len(alg_list_sst)):
#         temp = []
#         for rep_idx in range(repeats_sst):  # repeats_sst == 1 code
#             temp = stat_list_sst[n_idx][alg_idx][rep_idx].dual_gap_list[0]
#         temp_list.append(temp)
#     avg_dual_gap.append(temp_list)
#
# x_list_sst = np.arange(T_cap_sst / dual_gap_freq_sst) * dual_gap_freq_sst
#
# for n_idx in range(len(n_list_sst)):
#     plt.subplot(2, math.ceil(len(n_list_sst) / 2), n_idx + 1)
#     for k_idx in range(len(alg_list_sst)):
#         plt.plot(x_list_sst, avg_dual_gap[n_idx][k_idx], label=alg_name_sst[k_idx])
#         plt.ylabel('Duality Gap')
#         plt.xlabel('iterations')
#         plt.title('n = %s Duality Gap' % n_list_sst[n_idx])
#         plt.legend(loc="upper left")
#
# plt.show()

# In[20]:

#
# """
#
# Plot 3: Testing convergence on different K value
#
# Here we run for fixed T_cap, However we use optimal step-size for each case with very low epsilon 1e-8, so as to not
# terminate before T_cap.
#
# """
#
# # T = 100000 #Number of Total Iteration
# J_sst = 2  # Number of Cohorts
# L_sst = 5  # Number of Treatments
#
# opt_low_sst = 0  # lower bound of optimal value
# opt_up_sst = 2  # upper bound of optimal value
# obj_tol_sst = 1e-2  # tolerance of our objective value\\ tolerance of R_x and R_i, obj_tol and alg_tol should be same.
# C_K_sst = 0.7  # proportion of epsilon that we need to pay by using i_hat
# alpha_tol_sst = 1e-5
# dual_gap_freq_sst = 10  # We need to set this to one in order to track duality gap at every iteration
# # m = 3  # number of constraints, first index 0 refers to our objective function.
# # K = 100  # Sample size of our gradient estimator
# # K_grad = 1
# # N = 100  # size of each uncertainty set\\ We are no longer using this parameter in this code
# # n = 10 ** 2  # sample size of simulation data set, also support size of each uncertainty set.
#
# # nu = 0.1 # |g_t - h_t| <= nu
# # epsilon = 0.001 #epsilon-subgradient
#
#
# # Uncertainty set parameter
# delta_sst = 0.5  # p \geq \delta/n
# rho_sst = 0.01  # D_f(p,q) \leq \rho/n
# nu_sst = 0.1  # Prob of high convergence
#
# # emp_dist_value_1 = np.random.normal(loc=0.5, scale=0.125, size=(m, J, L - 1, n))
# # emp_dist_value_2 = np.random.normal(loc=0.25, scale=0.125, size=(m, J, 1, n))
# # emp_dist_value = np.concatenate([emp_dist_value_1, emp_dist_value_2], axis=2)
# # RHS = np.ones([m])
#
# # Get statistics of various cases.
#
#
# n_list_sst = [3000]
# K_list_sst = [1, 10, 100, 500, 1000, 1500, 'i_star']  # 6 Instance
# K_grad_sst = 1
# alg_list_sst = []
# T_cap_sst = 1000
# ss_type_sst = 'diminish'
# var_scale_sst = 0.1
#
# for k_idx in range(len(K_list_sst)):
#     temp = []
#     temp.append(K_list_sst[k_idx])
#     temp.append(K_grad_sst)
#     alg_list_sst.append(temp)
#
# repeats_sst = 1  # Currently we don't take average here
# m_sst = 8
#
# stat_list_sst = []  # stat_list_sst[n_idx][alg_idx][rep_idx]
#
# # Create random mean matrix for our data, [0,0.25] uniform distribution
# mean_array_sst = np.random.rand(m_sst, J_sst, L_sst) / J_sst
#
# for n_num in n_list_sst:
#     stat_alg = []
#
#     # Create our dataset
#     emp_dist_value = np.zeros([m_sst, J_sst, L_sst, n_num])
#     emp_dist_value[:] = np.nan
#     x_0_sst = np.ones([J_sst, L_sst]) / L_sst
#     p_0_sst = np.ones([m_sst, n_num]) / n_num
#     RHS_sst = np.ones([m_sst])  # RHS is important for determining K!
#
#     # #Simple data generation
#     # emp_dist_value_1 = np.random.normal(loc=0.5, scale=0.125, size=(m_sst, J_sst, int(L_sst / 2), n_num))
#     # emp_dist_value_2 = np.random.normal(loc=0.25, scale=0.125, size=(m_sst, J_sst, int((L_sst + 1) / 2), n_num))
#     # emp_dist_value = np.concatenate([emp_dist_value_1, emp_dist_value_2], axis=2)
#
#     # Random mean data generation
#     emp_dist_value = np.zeros([m_sst, J_sst, L_sst, n_num])
#     emp_dist_value[:] = np.nan
#     for m_idx in range(m_sst):
#         for j_idx in range(J_sst):
#             for l_idx in range(L_sst):
#                 emp_dist_value[m_idx, j_idx, l_idx, :] = np.random.normal(loc=mean_array_sst[m_idx, j_idx, l_idx],
#                                                                           scale=0.125, size=n_num)
#
#     # Get Approximated Obj Val
#     approx_obj_sst = 0
#     for j_idx in range(J_sst):
#         approx_obj_sst += np.max(mean_array_sst[0, j_idx, :])
#
#     print('approx_obj_sst:', approx_obj_sst)
#     opt_up_sst = 2 * approx_obj_sst
#
#     G_sst = np.absolute(emp_dist_value).max()
#     print('G:', G_sst)
#     M_sst = []
#     for i in range(m_sst):
#         M_sst.append(J_sst * np.absolute(emp_dist_value[i, :, :, :]).max())
#
#     for alg_idx in range(len(alg_list_sst)):
#         stat_repeat = []
#         K_sst = 0
#         K_grad_sst = 0
#
#         if alg_list_sst[alg_idx][0] == 'i_star':
#             K_sst = n_num
#         else:
#             K_sst = alg_list_sst[alg_idx][0]
#
#         if alg_list_sst[alg_idx][1] == 'FGD':
#             K_grad_sst = n_num
#         else:
#             K_grad_sst = alg_list_sst[alg_idx][1]
#
#         if K_sst == n_num:
#             C_K_sst = 1
#
#         random_samples_list = np.random.rand(T_cap_sst, m_sst, K_sst)
#
#         # Repeat on the same dataset we have
#         for rep_idx in range(repeats_sst):
#             print("K_sst:", K_sst)
#             alg_stat = DRO_Solver(x_0_sst, p_0_sst, emp_dist_value, K_sst, K_grad_sst, 'entropy', 'chi-square',
#                                   delta_sst, rho_sst, nu_sst, alpha_tol_sst, opt_low_sst, opt_up_sst,
#                                   obj_tol_sst, RHS_sst, ss_type_sst, random_samples_list, C_K_sst, \
#                                   dual_gap_freq_sst, T_cap_sst, print_option=0, K_test_flag=1, min_flag=0)
#
#             stat_repeat.append(alg_stat)
#         stat_alg.append(stat_repeat)
#     stat_list_sst.append(stat_alg)
#
# # Draw plot 3
#
# # stat_list_sst[n_idx][alg_idx or k_idx][rep_idx]
#
# # Draw some plot here.
# # We also need to consider early termination than T_cap!
#
#
# i_flag_count_list = []
#
# for n_idx in range(len(n_list_sst)):
#     temp_list = []
#     for alg_idx in range(len(alg_list_sst)):
#         for rep_idx in range(repeats_sst):  # repeats_sst == 1 code
#             temp = stat_list_sst[n_idx][alg_idx][rep_idx].i_flag_count
#         temp_list.append(temp)
#     i_flag_count_list.append(temp_list)
#
# for n_idx in range(len(n_list_sst)):
#     print('i flag count when n = %s' % n_list_sst[n_idx])
#     for k_idx, value in enumerate(i_flag_count_list[n_idx]):
#         print('K value: %s, i_flag_count: %s ' % (K_list_sst[k_idx], value))
#
# color_list_sst = []
# alg_name_sst = []
# for k_idx in range(len(K_list_sst)):
#     if K_list_sst[k_idx] != 'i_star':
#         alg_name_sst.append('K = ' + str(K_list_sst[k_idx]))
#     else:
#         alg_name_sst.append('K = Full Sample')
#
# avg_dual_gap = []  # avg_dual_gap[n_idx][k_idx]
#
# for n_idx in range(len(n_list_sst)):
#     temp_list = []
#     for alg_idx in range(len(alg_list_sst)):
#         temp = []
#         for rep_idx in range(repeats_sst):  # repeats_sst == 1 code
#             temp = stat_list_sst[n_idx][alg_idx][rep_idx].dual_gap_list[0]
#         temp_list.append(temp)
#     avg_dual_gap.append(temp_list)
#
# x_list_sst = np.arange(T_cap_sst / dual_gap_freq_sst) * dual_gap_freq_sst
#
# for n_idx in range(len(n_list_sst)):
#     plt.subplot(2, math.ceil(len(n_list_sst) / 2), n_idx + 1)
#     for k_idx in range(len(alg_list_sst)):
#         plt.plot(x_list_sst, avg_dual_gap[n_idx][k_idx], label=alg_name_sst[k_idx])
#         plt.ylabel('Duality Gap')
#         plt.xlabel('iterations')
#         plt.title('n = %s Duality Gap' % n_list_sst[n_idx])
#         plt.legend(loc="upper left")
#
# plt.show()